In [3]:
%tensorflow_version 1.x
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf



In [ ]:
# Set some parameters
IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 3

TRAIN_PATH = '/content/drive/My Drive/playWithData/train_set'
TEST_PATH = '/content/drive/My Drive/playWithData/test_set'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [4]:
train_ids = next(os.walk(TRAIN_PATH+'/'+'x_train'))[2]
train_ids_n = next(os.walk(TRAIN_PATH+'/'+'x_train_normal'))[2]

test_ids = next(os.walk(TEST_PATH+'/'+'x_test'))[2]
test_ids_n = next(os.walk(TEST_PATH+'/'+'x_test_normal'))[2]

print(len(train_ids_n), len(train_ids), len(test_ids), len(test_ids_n))

StopIteration: 

In [0]:
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')

sys.stdout.flush()

for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
  # print(n, id_)
  path = TRAIN_PATH + '/'+'x_train/'+id_
  img = imread(path)[:,:,:IMG_CHANNELS]
  img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
  X_train[n] = img
  mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
  mask_name = id_.split('.')[0]+'m.'+id_.split('.')[1]
  mask_ = imread(TRAIN_PATH+'/y_train/'+mask_name)
  
  mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
  # try:
  mask = np.maximum(mask, mask_)
  # except:
  #   print(TRAIN_PATH+'/y_train/'+mask_name)
  #   mask_ = imread(TRAIN_PATH+'/y_train/'+mask_name)
  #   print(mask_.shape)
  #   break
  Y_train[n]= mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')

sys.stdout.flush()

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + '/'+'x_test/'+id_
    img = imread(path)[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

In [0]:
rand = random.randint(0, len(train_ids))
imshow(X_train[rand])
plt.show()
imshow(np.squeeze(Y_train[rand]))
plt.show()

In [0]:
# Define IoU metric

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [0]:
#defining loss functions
def dice_loss(y_true, y_pred):
  numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
  denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))

  return 1 - numerator / denominator
def focal_loss(alpha=0.25, gamma=2):
  def focal_loss_with_logits(logits, targets, alpha, gamma, y_pred):
    weight_a = alpha * (1 - y_pred) ** gamma * targets
    weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)
    
    return (tf.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(-logits)) * (weight_a + weight_b) + logits * weight_b 

  def loss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
    logits = tf.log(y_pred / (1 - y_pred))

    loss = focal_loss_with_logits(logits=logits, targets=y_true, alpha=alpha, gamma=gamma, y_pred=y_pred)

    # or reduce_sum and/or axis=-1
    return tf.reduce_mean(loss)

  return loss

In [0]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

In [0]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('/content/drive/My Drive/model/model_nd_binary_large_more.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=50, 
                    callbacks=[earlystopper, checkpointer])

![picture](https://miro.medium.com/max/1400/1*OkUrpDD6I0FpugA_bbYBJQ.png)

![image](https://miro.medium.com/max/2000/1*yzbjioOqZDYbO6yHMVpXVQ.jpeg)

In [0]:
# Predict on train, val and test
model = load_model('/content/drive/My Drive/model/model_nd_binary_large_more.h5', custom_objects={'mean_iou': mean_iou})
pred_on_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
pred_on_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
pred_on_test = model.predict(X_test, verbose=1)

# Threshold predictions
pred_on_train_t = (pred_on_train > 0.5).astype(np.uint8)
pred_on_val_t = (pred_on_val > 0.5).astype(np.uint8)
pred_on_test_t = (pred_on_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
pred_on_test_upsampled = []
for i in range(len(pred_on_test)):
    pred_on_test_upsampled.append(resize(np.squeeze(pred_on_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

In [0]:
fig, axs = plt.subplots(1,3, figsize=(15, 15))
ix = random.randint(0, len(pred_on_train_t))
axs[0].set_title('Endoscopic Image')
axs[0].imshow(X_train[ix])

# plt.show()
axs[1].set_title('Vascular Location')
axs[1].imshow(np.squeeze(Y_train[ix]))
# plt.show()

axs[2].set_title('Predicted Vascular Location')
axs[2].imshow(np.squeeze(pred_on_train_t[ix]));
# plt.show()

In [0]:
ix = random.randint(0, len(pred_on_val_t))
fig, axs = plt.subplots(1,3, figsize=(15, 15))

axs[0].set_title('Endoscopic Image')
axs[0].imshow(X_train[int(X_train.shape[0]*0.9):][ix])

axs[1].set_title('Vascular Location')
axs[1].imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))

axs[2].set_title('Predicted Vascular Location')
axs[2].imshow(np.squeeze(pred_on_val_t[ix]));